# Project - Airline AI Assistant

We'll now bring together what we've learned to make an AI Customer Support assistant for an Airline

In [202]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage
from langchain_core.messages import ToolMessage
from langchain_core.tools import tool



In [226]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
api_key = os.getenv("GOOGLE_API_KEY_1")

if api_key:
    print(f"Google API Key exists and begins {api_key[:8]}")
else:
    print("Google API Key not set")

Google API Key exists and begins AIzaSyA6


In [227]:
MODEL = "gemini-2.0-flash"
model = ChatGoogleGenerativeAI(model=MODEL, api_key=api_key)

In [228]:
system_message = (
    "You are a helpful assistant for an airline called FlightAI. "
"Whenever the user asks for a ticket price, you **must call the 'get_ticket_price' function** to get the correct price. "
"Do not guess or make up prices. Give short, courteous answers in a natural way."
)

In [68]:
system_message

"You are a helpful assistant for an airline called FlightAI. Whenever the user asks about ticket prices, always call the 'get_ticket_price' function to get the correct price. Give short, courteous answers, no more than 1 sentence. If you don't know the answer, say so."

In [69]:
# This function looks rather simpler than the one from my video, because we're taking advantage of the latest Gradio updates

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    print(f"{messages}\n\n\n")
    response = model.invoke(messages)
    return response.content

gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7871
* To create a public link, set `share=True` in `launch()`.


## Tools

Tools are an incredibly powerful feature provided by the frontier LLMs.

With tools, you can write a function, and have the LLM call that function as part of its response.

Sounds almost spooky.. we're giving it the power to run code on our machine?

Well, kinda.

In [203]:
# Let's start by making a useful function

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}



def get_ticket_price(destination_city):
    """
    Python function that returns the price of a ticket to a given city.
    Get the price of a return ticket to the destination city.Call this whenever you need to know the ticket price, for example " "when a customer asks 'How much is a ticket to this city'
    Args:
        destination_city (str): Name of the city
    Returns:
        str: Ticket price or 'Unknown' if the city is not listed
    """
    print(f"[TOOL] get_ticket_price called with destination_city='{destination_city}'")
    city = destination_city.lower()  # Normalize input for dictionary lookup
    price = ticket_prices.get(city, "Unknown")
    print(f"[TOOL] Returning price: {price}")
    return price


In [71]:
get_ticket_price("Berlin")

[TOOL] get_ticket_price called with destination_city='Berlin'
[TOOL] Returning price: $499


'$499'

In [72]:
# -----------------------------
# 3️⃣ Tool Descriptor (Function Metadata)
# -----------------------------
# This tells the model about the tool: its name, description, and input arguments
price_function = {
    "name": "get_ticket_price",
    "description": (
        "Get the price of a return ticket to the destination city. "
        "Call this whenever you need to know the ticket price, for example "
        "when a customer asks 'How much is a ticket to this city'"
    ),
    "parameters": {
        "type": "object",  # input is a JSON object
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to"
            },
        },
        "required": ["destination_city"],  # city is required
    }
}


In [206]:
# Tools list that will be passed to the model
tools = [price_function]

## Getting OpenAI to use our Tool

There's some fiddly stuff to allow OpenAI "to call our tool"

What we actually do is give the LLM the opportunity to inform us that it wants us to run the tool.

Here's how the new chat function looks:

In [74]:
# # We have to write that function handle_tool_call:

# def handle_tool_call(message):
#     tool_call = message.tool_calls[0]
#     arguments = json.loads(tool_call.function.arguments)
#     city = arguments.get('destination_city')
#     price = get_ticket_price(city)
#     response = {
#         "role": "tool",
#         "content": json.dumps({"destination_city": city,"price": price}),
#         "tool_call_id": tool_call.id
#     }
#     return response, city

In [176]:
# def chat(message, history):
#     messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
#     response = model.invoke( messages=messages, tools=tools)

#     if response.choices[0].finish_reason=="tool_calls":
#         message = response.choices[0].message
#         response, city = handle_tool_call(message)
#         messages.append(message)
#         messages.append(response)
#         response = openai.chat.completions.create(model=MODEL, messages=messages)
    
#     return response.choices[0].message.content

In [255]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage, ToolMessage

def chat(user_input, history):
    print(f"\n[CHAT] User input: {user_input}")

    # Build conversation messages
    messages = [SystemMessage(content="You are a helpful assistant.")] # Assuming a system message
    for h in history:
        role = HumanMessage if h["role"] == "user" else AIMessage
        messages.append(role(content=h["content"]))
    messages.append(HumanMessage(content=user_input))

    # Bind tools and force usage
    model_with_tools = model.bind_tools(tools, tool_choice="any")
    response = model_with_tools.invoke(messages)

    # Check if the model made a tool call
    if tool_calls := getattr(response, "tool_calls", None):
        # --- Start of Changes ---

        # 1. Append the model's response (which contains the tool call) to the history
        messages.append(response)

        # 2. Execute each tool call and create a ToolMessage for each
        for tool_call in tool_calls:
            city = tool_call["args"]["destination_city"]
            price = get_ticket_price(city)
            print(f"[TOOL] Called get_ticket_price for '{city}', returned: {price}")
            
            # 3. Append the ToolMessage with the result and the original tool_call_id
            messages.append(ToolMessage(
                content=str(price), # The content should be the raw output of the tool
                tool_call_id=tool_call['id']
            ))

        # --- End of Changes ---

        # Call model again so it can respond naturally
        response = model.invoke(messages)
        print(f"[CHAT] Response is: {response.content}")

    return response.content

In [256]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7940
* To create a public link, set `share=True` in `launch()`.



[CHAT] User input: price to london?
[TOOL] get_ticket_price called with destination_city='london'
[TOOL] Returning price: $799
[TOOL] Called get_ticket_price for 'london', returned: $799
[CHAT] Response is: The price to London is $799.

[CHAT] User input: I'd like to go to Paris?
[TOOL] get_ticket_price called with destination_city='Paris'
[TOOL] Returning price: $899
[TOOL] Called get_ticket_price for 'Paris', returned: $899
[CHAT] Response is: The price to Paris is $899.

[CHAT] User input: and tokyo?
[TOOL] get_ticket_price called with destination_city='tokyo'
[TOOL] Returning price: $1400
[TOOL] Called get_ticket_price for 'tokyo', returned: $1400
[CHAT] Response is: The price to Tokyo is $1400.

[CHAT] User input: and berlin
[TOOL] get_ticket_price called with destination_city='Berlin'
[TOOL] Returning price: $499
[TOOL] Called get_ticket_price for 'Berlin', returned: $499
[CHAT] Response is: The price to Berlin is $499.

[CHAT] User input: and paris
[TOOL] get_ticket_price calle